# iManhunt — Mathematische & Theoretische Herleitung (SP1–SP3)

Umgebung: Anaconda, Dev-Env iManhunt
Stand: 2025-08-17
Hinweis: Dieses „Notebook“ ist rein konzeptionell: nur Markdown & LaTeX (kein Code).

## Inhaltsübersicht

1. SP1 — Graph & Attribute
2. SP2 — Trait-Sampler & Mapping
3. SP3 — Reward-Composer
4. Anhang — Symboltabelle

## SP1 — Graph & Attribute
### 1.1 Stadtmodell als Graph
Synthetische Stadt als gerichteter Graph \( G=(V,E) \).  
- **Knoten** \( v\in V \): Kreuzungen/POIs mit Koordinaten \( (x_v,y_v,z_v) \).  
- **Kanten** \( e=(v\!\to\!u)\in E \): begehbare Straßen-/Wege-Segmente.

Euklidische Kantenlänge:
$$
L_e \;=\; \big\| (x_v,y_v)-(x_u,y_u) \big\|_2 .
$$

Höhenmodell per synthetischem DEM \( z = f(x,y) \).  
**Steigung** je Kante:
$$
\mathrm{grade}_e \;=\; \frac{z_u - z_v}{L_e}, 
\qquad 
\mathrm{slope\%}_e \;=\; 100 \cdot \mathrm{grade}_e .
$$

### 1.2 Straßentypen & Basisschrittgeschwindigkeit
Kantentyp \( \mathrm{type}(e) \in \{\text{arterial},\text{residential},\text{footpath}\} \).  
Basisspeed:
$$
v_0(\text{arterial}) \;\ge\; v_0(\text{residential}) \;\ge\; v_0(\text{footpath}) .
$$



### 1.3 Geschwindigkeitsmodell mit Steigung
Fußbewegung (Baseline) mit Dämpfung bergauf und gekapptem Boost bergab:
$$
v_e \;=\; 
\begin{cases}
v_0 \, \exp\!\big(-\alpha\,\mathrm{grade}_e\big), & \mathrm{grade}_e \ge 0,\\[4pt]
v_0 \, \big(1 + \min\{\,b\cdot |\mathrm{grade}_e|,\; b_{\max}\,\}\big), & \mathrm{grade}_e < 0.
\end{cases}
$$
mit Schranken \( v_{\min}\le v_e \le v_{\max}=v_0(1+\hat b) \).



### 1.4 Reisezeitmetrik
**Reisezeit** auf Kante \( e \):
$$
\tau_e \;=\; \frac{L_e}{v_e}.
$$
Dies induziert die zeitgewichtete Distanz \( d_\tau(v,u) \) (kürzeste Pfadzeit über Gewichte \( \tau \)).  
**Isochronen** (erreichbar in Zeit \(t\)):
$$
\mathcal{R}(t) \;=\; \{\, w \in V \mid d_\tau(v_0,w) \le t \,\}.
$$

### 1.5 Graph-Matrizen
Adjazenz \( A \), Gradmatrix \( D \), Laplace-Operator \( L = D - A \).  
Zeitgewichtete Varianten via \( A_\tau \) (z. B. \(\kappa_e \propto 1/\tau_e\)).

### 1.6 Zeitdynamik (Semi-Markov / kontinuierlich)
Verweilzeitabhängige Übergänge ⇒ **Semi-Markov-Prozess**; alternativ kontinuierlich mit Generator \( Q \):
$$
Q_{vu} \;=\; r_{v\to u}, \quad
Q_{vv} \;=\; -\sum_{u\neq v} r_{v\to u}, 
\qquad
\frac{d}{dt}\, p_t \;=\; p_t\,Q.
$$
Raten \( r_{v\to u} \) werden aus (i) nutzenbasierter Wahl (Softmax) und (ii) Zeitskalierung \( \kappa \) kalibriert.

---

## SP2 — Trait-Sampler & Mapping

### 2.1 Trait-Vektor
Profil durch Traits
$$
\theta = (\text{age},\text{imp},\text{plan},\text{health},\text{topo},\text{risk},\text{add},\text{fam},\text{mot}).
$$

### 2.2 Verteilungen (Sampler)
- \(\text{age} \sim \mathrm{Triangular}(a,m,b)\)  
- \(\text{imp},\text{plan},\text{health},\text{topo},\text{risk} \sim \mathrm{Beta}(\alpha,\beta)\in[0,1]\)  
- \(\text{add},\text{fam} \sim \mathcal{N}(\mu,\sigma^2)\) (Clipping \([\!-1,1]\))  
- \(\text{mot} \sim \mathrm{Categorical}(p_1,\dots,p_M)\)

### 2.3 Normalisierung
Für \(\text{age}\) zentrierte Skala via Sigmoid:
$$
a^\star \;=\; \sigma\!\left(\frac{\text{age}-\mu_a}{s_a}\right) \in (0,1), 
\qquad \sigma(x)=\frac{1}{1+e^{-x}}.
$$

### 2.4 Mapping → RL-Hyperparameter
**Discount (Horizont):**
$$
\gamma(\theta) \;=\; \gamma_{\min} + (\gamma_{\max}-\gamma_{\min}) \cdot \big[a^\star \cdot \text{plan}\big].
$$
**Entropie/Exploration:**
$$
\alpha_{\mathrm{ent}}(\theta) \;=\; \alpha_0 + c_{\mathrm{imp}} \cdot \text{imp}.
$$
**Max-Speed (relativ zum Typ):**
$$
v_{\max}(\theta,\mathrm{type}) \;=\; v_0(\mathrm{type}) \cdot \big(1 + c_h(\text{health}-\bar h)\big)\cdot \big(1 - c_t\,\text{topo}\big).
$$

### 2.5 Mapping → Reward-Gewichte
**Steigungskosten:**
$$
w_{\mathrm{slope}}(\theta) \;=\; w_{\mathrm{slope},0}\cdot\big(\text{topo} + (1-\text{health})\big).
$$
**Detektionsrisiko:**
$$
w_{\mathrm{detect}}(\theta) \;=\; w_{\mathrm{detect},0}\cdot(1-\text{risk}).
$$
**POI-Term (gekappt):**
$$
w_{\mathrm{POI}}(\theta) \;=\; \mathrm{clip}\!\big(w_{\mathrm{POI},0}\cdot(\lambda_{\mathrm{add}}\text{add} + \lambda_{\mathrm{fam}}\text{fam}),\, -\bar w,\,\bar w\big).
$$

---

## SP3 — Reward-Composer

### 3.1 Gesamtstruktur
Stückweise zusammengesetzter Reward je Schritt:
$$
\begin{aligned}
R \;=\;& 
\; w_{\mathrm{dist}}\cdot \Delta \mathrm{RadialDist}
\;-\; w_{\mathrm{time}}\cdot \Delta t
\;-\; w_{\mathrm{slope}}\cdot \mathrm{EnergyCost} \\
&\;-\; w_{\mathrm{detect}}\cdot \mathrm{DetectRisk}
\;+\; w_{\mathrm{goal}}\cdot \mathrm{GoalPot}
\;+\; w_{\mathrm{POI}}\cdot(\mathrm{POI}_{\mathrm{attr}}-\mathrm{POI}_{\mathrm{rep}})
\;-\; w_{\mathrm{fatigue}}\cdot \mathrm{Fatigue}.
\end{aligned}
$$

### 3.2 Distanz- & Zeitkomponenten
Radialer Fortschritt (vom Start \(v_0\) oder zur Zielmenge \(\mathcal{G}\)):
$$
\Delta \mathrm{RadialDist} \;\approx\; d_\mathrm{rad}(s_{t+\Delta t}) - d_\mathrm{rad}(s_t).
$$
Zeitstrafe: \( \Delta t \) (oder Traversalzeit \(\tau_e\)).

### 3.3 Energiemodell (Steigung)
Mit \( \Delta z_e = z_u - z_v \) und horizontaler Arbeit:
$$
\mathrm{EnergyCost} \;=\; \kappa_1 \cdot L_e \;+\; \kappa_2 \cdot \max\{0, \Delta z_e\},
$$
typ. \( \kappa_2 \gg \kappa_1 \) (bergauf deutlich teurer).  
Alternativ indirekt über \(v_e\) → \(\tau_e\).

### 3.4 Detektionsrisiko
Checkpoints \( C_i \) mit Radius \( r_i \), Sensitivität \( s_i \). Für Position \(v\):
$$
\mathrm{DetectRisk}(s_t) \;=\; \sum_i s_i \cdot \mathbf{1}\{\,v\in B_i(r_i)\,\}\cdot g(\text{Straßentyp}, t),
$$
mit Sichtbarkeitsmodulator \(g\) (z. B. Tageszeit).  
Begegnungswahrscheinlichkeit in Fenster \( \Delta t \) (näherungsweise unabhängig):
$$
P_{\mathrm{enc}}(t,\Delta t) \;=\; 1 - \prod_i \big(1 - q_i(t)\big), 
\qquad
q_i(t)=s_i \sum_{v\in B_i(r_i)} p_t(v).
$$

### 3.5 Ziel-Potentialfeld
Für Motiv \( \text{mot} \) Potential \( \Phi_{\text{mot}}:V\to\mathbb{R} \), meist fallend mit \( d_\tau(v,\mathcal{G}) \).  
Dann \( \mathrm{GoalPot} = -\,\Phi_{\text{mot}}(v) \).

### 3.6 POI-Attraktion/Repulsion
POI-Mengen \( \mathcal{P}^+ \) (Attraktion) und \( \mathcal{P}^- \) (Repulsion):
$$
\mathrm{POI}_{\mathrm{attr}}(v) \;=\; \sum_{p\in \mathcal{P}^+} k^+\,\psi\!\big(d_\tau(v,p)\big), \qquad
\mathrm{POI}_{\mathrm{rep}}(v) \;=\; \sum_{p\in \mathcal{P}^-} k^-\,\psi\!\big(d_\tau(v,p)\big),
$$
mit fallender Kernfunktion \( \psi(r)=e^{-\lambda r} \).  
Kappen \( |\mathrm{POI}_{\cdot}| \le M \) gegen extreme Effekte.

### 3.7 Ermüdung (Fatigue) & Geschwindigkeitskopplung
Fatigue \( F_t \) wächst mit Arbeit und erholt sich:
$$
F_{t+\Delta t} \;=\; F_t \;+\; \eta\cdot \mathrm{EnergyCost} \;-\; \rho\cdot \Delta t,\qquad F_t\ge 0.
$$
Effektive Geschwindigkeit:
$$
v_e^{(\mathrm{eff})} \;=\; \frac{v_e}{1+\lambda_F\,F_t},
$$
⇒ längere \(\tau_e\) und Rückkopplung Pfadwahl↔Leistungsfähigkeit.

### 3.8 Regularisierung & Caps
- \(|w_{\mathrm{POI}}|\), \(|\mathrm{POI}_{\cdot}|\) begrenzen.  
- Schranken auf \(v_e^{(\mathrm{eff})}\), Minimal-\(\tau_e\).  
- Entropie-Bonus (MaxEnt-RL) für robustere Policies.

---

## Anhang — Symboltabelle

| Symbol | Bedeutung |
|---|---|
| \(G=(V,E)\) | Graph (Knoten/Kanten) |
| \(L_e\) | Kantenlänge |
| \(\mathrm{grade}_e, \mathrm{slope\%}_e\) | Steigungsmaß |
| \(v_0, v_e, v_e^{(\mathrm{eff})}\) | Basis-, Kanten-, effektive Geschwindigkeit |
| \(\tau_e\) | Reisezeit auf Kante \(e\) |
| \(d_\tau\) | Zeitgewichtete Distanz |
| \(\mathcal{R}(t)\) | Isochrone |
| \(A,D,L\) | Adjazenz-, Grad-, Laplacian-Matrix |
| \(Q\) | Generator (kontinuierliche Zeit) |
| \(p_t\) | Aufenthaltsverteilung |
| \(\theta\) | Trait-Vektor |
| \(\gamma,\alpha_{\mathrm{ent}}\) | Discount, Entropiekoeff. |
| \(w_{\cdot}\) | Reward-Gewichte |
| \(R\) | Reward je Schritt |
| \(F_t\) | Ermüdung |
| \(C_i, r_i, s_i\) | Checkpoint, Radius, Sensitivität |
| \(q_i(t), P_{\mathrm{enc}}(t,\Delta t)\) | Begegnungswahrscheinlichkeiten |